In [89]:
import numpy as np
import pandas as pd
import jax.numpy as jnp
from jax import grad, jit, vmap
from sklearn.model_selection import train_test_split

In [90]:
from functions import AdamOptim, SimpleOptim

In [91]:
class Node:
    """
    Class for Node with split
    """
    def __init__(
        self,
        status=None,
        root=None,
        index=None,
        max_depth=None,
        current_f1=None
        
    ):
        
        # Saving the hyperparameters
        self.max_depth = max_depth if max_depth else 5
        
        # Setting index of elemnet
        self.index = index if index else 0
        self.depth = int(np.floor(np.log2(self.index + 2)))
        self.status = status if status else "node"
    
        # Init root with None
        self.root = root
        
    def init_node(self, index):
        # Init childs with Leafs
        self.l_child = \
            Node(
                status='suboptimal_leaf',\
                root=self.root,\
                index=2*index + 1, \
                max_depth=self.max_depth)
        self.r_child = \
            Node(
                status='suboptimal_leaf',\
                root=self.root,\
                index=2*index + 2,\
                max_depth=self.max_depth)
        
        
    
    def predict(self, 
                observations: list,
                node_parameters):

        current = self
        index = 0
        product_path = 1
        l1_norm_sum = 0
        
        # проходим по дереву пока не упираемся в leaf
        while current.status == 'node':
            # use formula for probabilty
            probabilty = \
                self.sigmoid(jnp.dot(node_parameters[index], \
                jnp.concatenate((observations, 1), axis=None)))
            l1_norm_sum += jnp.sum(jnp.abs(node_parameters[index]))

            
            if probabilty > 0.5:
#                 print('RIGHT')
                product_path = jnp.multiply(product_path, probabilty)
                current = current.l_child
                index = 2*index + 1
            else:
#                 print('LEFT')
                product_path = jnp.multiply(product_path, 1 - probabilty)
                current = current.r_child
                index = 2*index + 2
                
            
                
        # возвращаем предсказанное действие, следующую историю, и предсказания относительно
        # будущих наблюдений        
#         return (jnp.tanh(action_parameters[index]), index, jnp.tanh(pred_observation[index]), product_path)
        return index, product_path, l1_norm_sum               
    
    def sigmoid(self, value):
        return 1 / (1 + jnp.exp(-value))
    
    def copy(self):
        
        
        copy = Node(status=self.status,root=self.root,index=self.index,max_depth=self.max_depth )
        return copy



In [92]:
def init_root():
    root = Node()
    root.root = root
    root.init_node(root.index)
    return root

In [93]:
root = init_root()

In [94]:
root.l_child = Node(root=root,index=root.l_child.index )
root.l_child.init_node(root.l_child.index)

In [95]:
root.r_child = Node(root=root,index=root.r_child.index )
root.r_child.init_node(root.r_child.index)

In [96]:
root.r_child.r_child = Node(root=root,index=root.r_child.r_child.index )
root.r_child.r_child.init_node(root.r_child.r_child.index)

In [97]:
def softmax(vector):
    sum_ = jnp.sum(jnp.exp(vector))
    return jnp.exp(vector)/sum_

In [98]:
length_observation = 10
max_depth = 5
node_parameters = 2 * np.random.sample((2**max_depth - 1,  length_observation + 1)) - 1
action_parameters = 2 * np.random.sample((2**max_depth - 1, 2)) - 1

In [99]:
action_parameters[0]

array([0.41039844, 0.422499  ])

In [100]:
node_parameters[0]

array([-0.56401026, -0.94788279,  0.85295515,  0.57811035,  0.06506044,
        0.44796156, -0.01605264, -0.88330503,  0.18123098, -0.00504791,
       -0.47465759])

In [101]:
first_observation = 2 * np.random.sample(length_observation) - 1
index, product_path, l1_norm_sum = \
                root.predict(first_observation,
                node_parameters)



In [102]:
l1_norm_sum

DeviceArray(9.909757, dtype=float32)

In [103]:
action_parameters[index]

array([ 0.98975721, -0.51649003])

In [104]:
def to_actions(vector):
    actions = {0: 'not fraud', 1: 'fraud'}
#     print(jnp.argmax(vector))
    return actions[int(jnp.argmax(vector))]

In [105]:
def process(element, node_parameters, action_parameters):
#     print(np.log2(element.index + 1))
    print('\t'*int(np.floor(np.log2(element.index+1))) + str(element.index) + ' ' + element.status, end='')
    if element.status!='node':
#         print('\t'*int(np.floor(np.log2(element.index+1))) , end='')
        print('\t', end='')
        to_actions(action_parameters[element.index])
        print(to_actions(action_parameters[element.index]))
        print()
#         print(action_parameters[element.index])
    if element.status=='node':
        print(': some split')
        process(element.l_child, node_parameters, action_parameters)
        process(element.r_child, node_parameters, action_parameters)
    

In [106]:
process(root, node_parameters, action_parameters)

0 node: some split
	1 node: some split
		3 suboptimal_leaf	not fraud

		4 suboptimal_leaf	fraud

	2 node: some split
		5 suboptimal_leaf	fraud

		6 node: some split
			13 suboptimal_leaf	not fraud

			14 suboptimal_leaf	fraud



In [107]:

def loss_function(node_parameters,
                    action_parameters,
                    root,
                    observation, 
                    actions,
                    reg_coef):
    
#     predicted_actions, next_history_predicted_index, pred_obs, product_path = \
    index, product_path, l1_norm_sum = \
        root.predict(observation,
                    node_parameters)
    predicted_actions = action_parameters[index]
    general_loss = - jnp.dot(product_path, jnp.sum(actions * jnp.log(softmax(predicted_actions))))
    return general_loss + l1_norm_sum * reg_coef
            

In [108]:
first_observation = 2 * np.random.sample(length_observation) - 1
actions = jnp.array([0, 1])
reg_coef = 0.01
loss_function(node_parameters,
                action_parameters,
                root,
                first_observation,
                actions,
                reg_coef)

DeviceArray(0.39431792, dtype=float32)

In [109]:
grad_loss_node = grad(loss_function, 0)

In [110]:
grad_loss_node(node_parameters,
                action_parameters,
                root,
                first_observation,
                actions,
                reg_coef)


DeviceArray([[-9.2701418e-03, -6.1569914e-02,  2.5905669e-05,
               6.2038630e-02,  8.0850393e-02, -5.0020203e-02,
              -7.4635595e-02,  2.9567586e-02,  2.8827721e-02,
              -2.7523698e-02, -8.5838057e-02],
             [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00],
             [-9.6484339e-03, -1.4840724e-02, -1.4804424e-02,
               3.5066500e-02,  4.4127941e-02, -1.8911147e-02,
              -4.1134335e-02,  2.9059321e-02, -9.3086995e-04,
              -1.8440995e-02, -4.6530455e-02],
             [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
               0.0000000e+00,  0.0000000e+00],
             [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
      

In [111]:
df = pd.read_csv('processed dataset.csv', sep='\t')
df = df.drop(['Unnamed: 0'], axis=1)

In [112]:
df = df.sample(frac=1)

In [113]:
df.head()

,CARD_EXPR_DATE,AMOUNT,AMOUNT_INST,DATE_TIME,PDC_CARD_PRES,PDC_CARD_CAPTURE_CAP,CVV2_PRES,AUTH_RESP_CODE_FM,PIN_PRES,MARKED_DATE
19534,2.029863,0.000137,0.004330,0.005556,0,1,1,1,0,0
26101,2.227603,0.001353,0.048027,0.021111,0,0,1,1,0,0
46718,1.921913,0.002101,0.274617,0.064444,0,0,1,1,0,1
51372,2.130751,0.000005,0.000183,0.012222,0,0,0,1,0,0
40414,2.021792,0.000350,0.000893,0.041944,0,0,0,1,0,0


In [114]:
train_df, test_and_val_df = train_test_split(df, test_size=0.5)
val_df, test_df = train_test_split(df, test_size=0.4)

In [115]:
train_df.shape

(32499, 10)

In [116]:
train_size = train_df.shape[0]

In [117]:
not_fraud_size = (df['MARKED_DATE'] == 0).sum()

In [118]:
fraud_size = (df['MARKED_DATE'] == 1).sum()

In [119]:
# lets oversample!

In [120]:
fraud = df[df['MARKED_DATE'] == 1]

In [121]:
fraud.head()

,CARD_EXPR_DATE,AMOUNT,AMOUNT_INST,DATE_TIME,PDC_CARD_PRES,PDC_CARD_CAPTURE_CAP,CVV2_PRES,AUTH_RESP_CODE_FM,PIN_PRES,MARKED_DATE
46718,1.921913,0.002101,0.274617,0.064444,0,0,1,1,0,1
2535,2.130751,0.000000,0.000000,0.007500,0,0,1,1,0,1
51779,1.821025,0.001049,0.137074,0.002222,0,0,1,1,0,1
20046,2.433414,0.000004,0.000177,0.002500,0,0,1,1,0,1
32697,2.029863,0.000002,0.000291,0.000278,0,0,0,1,0,1


In [122]:

train_df_oversampled = train_df.sample(frac=1)
num_of_sample = 100
proportion = 0.2
b1 = not_fraud_size * proportion  / (1 - proportion) - fraud_size
# print(b1)
for i in range(int(np.floor(b1 / num_of_sample))):
    train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))

/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_s

/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_s

/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_s

/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_sample))
/tmp/ipykernel_4598/1705457642.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df_oversampled = train_df_oversampled.append(fraud.sample(num_of_s

In [123]:
len(train_df_oversampled)

45699

In [124]:
train_df.head()

,CARD_EXPR_DATE,AMOUNT,AMOUNT_INST,DATE_TIME,PDC_CARD_PRES,PDC_CARD_CAPTURE_CAP,CVV2_PRES,AUTH_RESP_CODE_FM,PIN_PRES,MARKED_DATE
44917,2.123688,0.000242,0.031675,0.010000,0,0,0,1,0,0
45142,2.225585,0.000107,0.003826,0.035278,0,0,1,1,0,0
29463,1.819007,0.000119,0.005625,0.016111,0,1,1,1,0,0
58136,1.928975,0.000174,0.022691,0.019722,0,0,1,1,0,0
40198,2.123688,0.000312,0.013014,0.000000,0,1,1,1,0,1


In [125]:
train_df.tail()

,CARD_EXPR_DATE,AMOUNT,AMOUNT_INST,DATE_TIME,PDC_CARD_PRES,PDC_CARD_CAPTURE_CAP,CVV2_PRES,AUTH_RESP_CODE_FM,PIN_PRES,MARKED_DATE
10411,1.928975,8.410980e-06,0.000301,0.011389,0,0,0,1,0,0
37627,2.130751,2.577124e-04,0.009230,0.013056,1,0,-1,1,0,0
35685,2.229621,3.504575e-06,0.000457,0.001389,1,0,-1,1,0,0
19135,2.227603,7.009150e-07,0.000030,0.003333,0,0,0,1,0,0
33637,1.919895,3.504575e-05,0.001255,0.000278,0,0,1,1,0,0


In [126]:
train_df_oversampled = train_df_oversampled.sample(frac=1)

In [127]:
columns_names = train_df.columns

In [128]:
(train_df_oversampled['MARKED_DATE'] == 1).sum()

14389

In [129]:
(train_df_oversampled['MARKED_DATE'][1:100] == 1).sum() / len(train_df_oversampled[1:100])

0.3333333333333333

In [130]:
# train_df.head(15)

In [131]:
batch_actions = train_df_oversampled['MARKED_DATE'].values
batch_actions

array([0, 0, 0, ..., 1, 0, 0])

In [132]:
batch_observation = train_df_oversampled.drop(['MARKED_DATE'], axis=1).values
batch_observation

array([[2.12368846e+00, 7.00914960e-09, 2.51042851e-07, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.22356739e+00, 9.78477285e-06, 3.50455820e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.13075061e+00, 1.02352509e-03, 1.33439066e-01, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [2.12368846e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.12368846e+00, 7.00914960e-06, 2.51042851e-04, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.02683616e+00, 1.87011121e-04, 6.69807430e-03, ...,
        1.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [133]:
len(train_df)/100

324.99

In [134]:

class Generator:
    def __init__(self, batch_observation, batch_actions, batch_size):
        self.batch_observation = batch_observation
        self.batch_actions = batch_actions
        self.batch_size = batch_size
        self.split = 0
        self.num_of_rows = len(batch_observation)
    def get_batch(self):
#         print((self.split + self.batch_size))
#         print(self.num_of_rows)
        if (self.split + self.batch_size) < self.num_of_rows:
            sample_observations = self.batch_observation[self.split : self.split + self.batch_size]
            true_actions = self.batch_actions[self.split : self.split + self.batch_size]

            sample_actions = np.zeros((true_actions.size, 2))
            sample_actions[np.arange(true_actions.size),true_actions] = 1
            self.split += self.batch_size
        else:
            return "is_full"
        
        
        return (jnp.array(sample_observations), jnp.array(sample_actions))

In [135]:
batch_size = 100
generator = Generator(batch_observation, batch_actions, batch_size)

In [136]:
sample_observations, sample_actions = generator.get_batch()

In [137]:
np.argmax(sample_actions, axis=1)

DeviceArray([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
             0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
             1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
             0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
             0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1],            dtype=int32)

In [138]:
sample_observations.shape

(100, 9)

In [139]:
# sample_observations

In [140]:

    

def process_batch(root,
                  batch_observation,
                  batch_actions,
                  length_observation,
                  max_depth,
                  node_parameters,
                  action_parameters,
                  reg_coef):
    
    node_params_grads = np.zeros((2**(max_depth - 1 )-1, length_observation + 1))
    action_params_grads = np.zeros((2**max_depth -1, 2))
    cumulative_loss = 0

    current_index = 0
    
    predictions = np.array([])
    
    for i in range(len(batch_observation)):
        cumulative_loss += \
            loss_function(node_parameters, \
                action_parameters, \
                root, \
                batch_observation[i], \
                batch_actions[i], \
                reg_coef)
        
#         grad_f =
        
        node_grad =  grad(loss_function, 0)(node_parameters,
                                   action_parameters,
                                   root,
                                   batch_observation[i],
                                   batch_actions[i],
                                   reg_coef)
#         print(batch_observation[i].shape)
#         print(batch_actions[i].shape)
#         print(node_params_grads.shape)
#         print(node_grad.shape)
#         print(node_parameters.shape)
#         print(action_parameters.shape)
#         node_params_grads += node_grad
            
        action_params_grads += \
            grad(loss_function, 1)(node_parameters,
                                   action_parameters,
                                   root,
                                   batch_observation[i],
                                   batch_actions[i],
                                   reg_coef)

        current_index, product_path, l1_norm_sum = \
            root.predict(batch_observation[i],
                    node_parameters)
        predictions = np.append(predictions,
                               np.argmax(\
                                softmax(\
                                action_parameters[current_index])))
        
        
    return (node_params_grads, 
            action_params_grads,
            cumulative_loss,
            predictions)

In [141]:
# sample_actions

In [142]:
length_observation = 9
max_depth = 4
node_parameters = 1 * np.random.sample((2**max_depth - 1,  length_observation + 1)) - 2
action_parameters = 1 * np.random.sample((2**max_depth - 1, 2)) - 2
reg_coef = 0.00001

gradients = process_batch(root,
              sample_observations, 
              sample_actions,
              length_observation,
              max_depth,
              node_parameters,
              action_parameters,
                reg_coef)

In [143]:
# node_parameters,
# action_parametrs gradientsers,
gradients[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [144]:
gradients[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [145]:
for i in range(int(np.floor(len(train_df) / batch_size))):
    answer = generator.get_batch()
    if answer == 'is_full':
        break
    else:
        sample_observations, sample_actions = answer[0], answer[1]
#         print(len(sample_observations))

In [146]:

def count_metrics(predictions, true_values):
    N = true_values.shape[0]
    accuracy = np.sum(true_values == predictions)/ N
    TP = ((predictions == 1) & (true_values == 1)).sum()
    FP = ((predictions == 1) & (true_values == 0)).sum()
    FN = ((predictions == 0 ) & (true_values == 1 )).sum()
    
    if (TP+FP) == 0:
        precision = 0
    else:
        precision = TP / (TP+FP)
        
    if (TP+FN) == 0:
        recall = 0
    else:
        recall = TP/(TP+FN)
        
    if recall == 0 or precision == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
        
        
    return accuracy, precision, recall, f1

In [147]:
# def train_adam_optim(train_df, length_observation, batch_size, max_depth, reg_coef, eta, beta1, beta2):
#     node_parameters = 2 * np.random.sample((2**max_depth - 1,  length_observation + 1)) - 1
#     action_parameters = 2 * np.random.sample((2**max_depth - 1, 2)) - 1
#     train_df = train_df.sample(frac=1)
    
#     batch_actions = train_df['MARKED_DATE'].values
#     batch_observation = train_df.drop(['MARKED_DATE'], axis=1).values
    
# #     print(np.sum()
    
#     generator = Generator(batch_observation, batch_actions, batch_size)
#     adam = AdamOptim(length_observation=length_observation, \
#                      max_depth=max_depth,\
#                      eta=eta,\
#                      beta1=beta1,\
#                      beta2=beta2)
    
#     data = pd.DataFrame({'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 
    
# #     for i in range(int(np.floor(len(train_df) / (batch_size)))):
#     for i in range(50):
# #     for i in range(5):
#         sample_observations, sample_actions = generator.get_batch()
        
#         gradients = process_batch(sample_observations, 
#                       sample_actions, 
#                       length_observation,
#                       max_depth,
#                       node_parameters,
#                       action_parameters,
#                         reg_coef)
          
#         node_parameters, action_parameters = \
#             adam.update(i + 1,np=node_parameters,\
#                         ap=action_parameters, dnp=gradients[0], dap=gradients[1])
# #         node_parameters -= 0.01 * gradients[0]
# #         action_parameters -= 0.01 * gradients[1]
        
#         predictions = gradients[3]
#         accuracy, precision, recall, f1 = count_metrics(predictions, np.argmax(sample_actions, axis=1))
# #         print(predictions.shape)
# #         print( np.argmax(sample_actions, axis=1).shape)
#         print('Epoch: ', i, '\t',end='')
    
#         print(' average_loss = %.2f' % (gradients[2] / batch_size), '\t', end='')
# #         print(' accuracy = %.3f' % accuracy, '\t', end='')
        
# #         print(' num_positive_pred: %d'% int(np.sum(predictions) ),'\t', end='')
        
# #         print(' num_positive_true: %d' % int(np.sum(np.argmax(sample_actions, axis = 1))),'\t',end='')
        
# #         print(np.argmax(sample_actions))
        
#         if precision == 'inf':
#             print(' inf', '\t', end='')
#         else:
#              print(' precision = %.5f' % precision, '\t', end='')
        
# #         if recall == 'inf':
# #             print(' inf','\t', end='')
# #         else:
# #              print(' recall = %.5f' % recall, '\t', end='')
        
#         if f1 == 'inf':
#             print(' inf')
#         else:
#             print(' f1 = %.5f' % f1)
            
#         data.loc[len(data)] = [accuracy, precision,recall, f1]
        
#     return (node_parameters, action_parameters, data)


In [213]:


def train_simple_optim(train_df,\
                       root,\
                       node_parameters,\
                       action_parameters,\
                       num_epochs,\
                       length_observation,\
                       batch_size,\
                       max_depth,\
                       reg_coef,\
                       lr_np,\
                       lr_ap,\
                       theta,\
                       node_index):
    
    
    
#     node_parameters = 2 * np.random.sample((2**max_depth - 1,  length_observation + 1)) - 1
#     action_parameters = 2 * np.random.sample((2**max_depth - 1, 2)) - 1
    train_df = train_df.sample(frac=1)
    
    batch_actions = train_df['MARKED_DATE'].values
    batch_observation = train_df.drop(['MARKED_DATE'], axis=1).values
    
#     print(np.sum()
    
    generator = Generator(batch_observation, batch_actions, batch_size)
    simple = SimpleOptim(max_depth=max_depth,\
                     lr_np=lr_np,\
                     lr_ap=lr_ap,
                     theta=theta)
    
    data = pd.DataFrame({'loss':[],'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 
    print('-'*50)
#     for i in range(int(np.floor(len(train_df) / (batch_size)))):
    for i in range(num_epochs):
#     for i in range(5):
        sample_observations, sample_actions = generator.get_batch()
        
        gradients = process_batch(root,
                      sample_observations, 
                      sample_actions, 
                      length_observation,
                      max_depth,
                      node_parameters,
                      action_parameters,
                        reg_coef)
        
        dnp = gradients[0]
        dap = gradients[1]
        
#         print(dnp)
#         print(node_index)
#         print(dap)
        if not node_index is None:
            dnp_zeros = np.zeros(dnp.shape)
            dap_zeros = np.zeros(dap.shape)

            dnp_zeros[node_index] = dnp[node_index]
            
            dap_zeros[2*node_index + 1] = dap[2*node_index + 1]
            dap_zeros[2*node_index + 2] = dap[2*node_index + 2]
            
            dnp = dnp_zeros
            dap = dap_zeros
        
#         print(dnp)
#         print(dap)
        
        node_parameters, action_parameters = \
            simple.update(np=node_parameters,\
                        ap=action_parameters, dnp=dnp, dap=dap, t=i)
#         node_parameters -= 0.01 * gradients[0]
#         action_parameters -= 0.01 * gradients[1]
        
        predictions = gradients[3]
        accuracy, precision, recall, f1 = count_metrics(predictions, np.argmax(sample_actions, axis=1))
#         print(predictions.shape)
#         print( np.argmax(sample_actions, axis=1).shape)
        print('Epoch: ', i, '\t',end='')
    
        print(' average_loss = %.2f' % (gradients[2] / batch_size), '\t', end='')
        print(' accuracy = %.3f' % accuracy, '\t', end='')
        
#         print(' num_positive_pred: %d'% int(np.sum(predictions) ),'\t', end='')
        
#         print(' num_positive_true: %d' % int(np.sum(np.argmax(sample_actions, axis = 1))),'\t',end='')
        
#         print(np.argmax(sample_actions))
        
 
        print(' precision = %.5f' % precision, '\t', end='')
 
        print(' f1 = %.5f' % f1)
    
        
            
        data.loc[len(data)] = [gradients[2], accuracy, precision,recall, f1]
        
    return (node_parameters, action_parameters, data)

In [149]:
# node_parametebeta2sample_actionsrs, action_parameters, data = \
#     train(train_df,
#         root,
#         length_observation=9, 
#         batch_size=100,   
#         max_depth=3,
#         reg_coef=0.001,
#         eta=0.01,
#         beta1=0.9, 
#         beta2=0.999)

In [150]:
num_of_rounds = 50

In [151]:
batch_size_field = []
for i in range(num_of_rounds):
    r = np.random.randint(3,9)
    batch_size_field.append(2**r)
    
reg_coef_field = []
for i in range(num_of_rounds):
    r = np.random.randint(0,6)
    reg_coef_field.append(10**(-r))
    
eta_field = []
for i in range(num_of_rounds):
    r = np.random.randint(0,4)
    eta_field.append(10**(-r))
    
beta1_field = []
for i in range(num_of_rounds):
    r = np.random.randint(1,4)
    beta1_field.append(1 - 10**(-r))

beta2_field = []
for i in range(num_of_rounds):
    r = np.random.randint(1,4)
    beta2_field.append(1 - 10**(-r))
    
lr_np_field = []
for i in range(num_of_rounds):
    r = np.random.randint(1,4)
    lr_np_field.append(10**(-r))

lr_ap_field = []
for i in range(num_of_rounds):
    r = np.random.randint(1,4)
    lr_ap_field.append(10**(-r))
    
theta_field = []
for i in range(num_of_rounds):
    r = np.random.randint(0,4)
    theta_field.append(10**(-r))    

In [152]:
# lr_ap_field

In [153]:
val_df.shape

(38999, 10)

In [154]:
batch_observation_validation = val_df.drop(['MARKED_DATE'], axis=1).values
batch_actions_validation = val_df['MARKED_DATE'].values

In [155]:
# node_parameters = 2 * np.random.sample((2**max_depth - 1,  length_observation + 1)) - 1
# action_parameters = 2 * np.random.sample((2**max_depth - 1, 2)) - 1

    
# predictions = np.array([])
# for i in range(len(val_df)):
#     current_index, _, _ = \
#                     root.predict(batch_observation_validation[i],
#                     node_parameters)
#     predictions = np.append(predictions,
#                            np.argmax(\
#                             softmax(\
#                             action_parameters[current_index])))

# accuracy, precision, recall, f1 = count_metrics(predictions, batch_actions_validation)


In [156]:
# best_precision = 0.
# # history = pd.DataFrame({'bs':[], 'reg_coef':[], 'eta':[], 'beta1':[],'beta2':[],\
# #                         'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 
# history = pd.DataFrame({'bs':[], 'reg_coef':[], 'lrnp':[], 'lrap':[], 'theta': [],\
#                         'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 
# for i in range(num_of_rounds):
    
# #     node_parameters, action_parameters, data = \
# #     train_adam_optim(train_df, 
# #         length_observation=9, 
# #         batch_size=batch_size_field[i], 
# #         max_depth=4,
# #         reg_coef=reg_coef_field[i],
# #         eta=eta_field[i],
# #         beta1=beta1_field[i], 
# #         beta2=beta2_field[i])
    
#     node_parameters, action_parameters, data = \
#     train_simple_optim(train_df, 
#         length_observation=9, 
#         batch_size=batch_size_field[i], 
#         max_depth=4,
#         reg_coef=reg_coef_field[i],
#         lr_np=lr_np_field[i], 
#         lr_ap=lr_ap_field[i],
#         theta=theta_field[i])
    
    
#     predictions = np.array([])
#     for j in range(len(val_df)):
#         current_index, _, _ = \
#                         root.predict(batch_observation_validation[j],
#                         node_parameters)
#         predictions = np.append(predictions,
#                                np.argmax(\
#                                 softmax(\
#                                 action_parameters[current_index])))
        
        
# #     print(predictions)
# #     print(batch_actions_validation)
#     accuracy, precision, recall, f1 = count_metrics(predictions, batch_actions_validation)
    
#     if precision != 'inf':
#         if best_precision < precision:
#             best_accuracy = accuracy
#             best_recall = recall
#             best_precision = precision
#             best_f1 = f1
#             best_parameters = (node_parameters, action_parameters)
    
#     name = 'bs_' + str(batch_size_field[i]) + \
#     '_reg_coef_' + str(reg_coef_field[i]) + \
#     '_lrnp_' + str(lr_np_field[i]) + '_lrap_' + \
#     str(lr_ap_field[i]) + '_theta_' + str(theta_field[i]) + '.csv'

#     data.to_csv('log_2/' + name, sep='\t')
# #     new_line = \
# #         [batch_size_field[i], reg_coef_field[i], eta_field[i], \
# #          beta1_field[i], beta2_fie        rint('-'*50)
    
#     new_line = \
#         [batch_size_field[i], reg_coef_field[i], lr_np_field[i], \
#          lr_ap_field[i], theta_field[i], accuracy, precision, recall, f1]

#     history.loc[len(history)] = new_line
#     print(new_line)
#     history.to_csv('history.csv', sep='\t')  
    


In [157]:
# history_of_best_hp = pd.DataFrame({'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 

# num_of_rounds = 15
# for i in range(num_of_rounds):
    
#     node_parameters, action_parameters, data = \
#     train_simple_optim(train_df, 
#         length_observation=9, 
#         batch_size=128, 
#         max_depth=4,
#         reg_coef=0.001,
#         lr_np=0.001, 
#         lr_ap=0.010,
#         theta=0.010)
    
    
#     predictions = np.array([])
#     for j in range(len(val_df)):
#         current_index, _, _ = \
#                         root.predict(batch_observation_validation[j],
#                         node_parameters)
#         predictions = np.append(predictions,
#                                np.argmax(\
#                                 softmax(\
#                                 action_parameters[current_index])))
        
        
# #     print(predictions)
# #     print(batch_actions_validation)
#     accuracy, precision, recall, f1 = count_metrics(predictions, batch_actions_validation)
    
    
#     name = str(i) + '.csv'

#     data.to_csv('log_3/' + name, sep='\t')
    
#     new_line = \
#         [accuracy, precision, recall, f1]

#     history_of_best_hp.loc[len(history_of_best_hp)] = new_line
#     print(new_line)
#     history_of_best_hp.to_csv('history.csv', sep='\t')  
    


In [158]:
# border = 1000
# val_df = val_df[:border]
# batch_observation_validation = batch_observation_validation[:border]
# batch_actions_validation = batch_actions_validation[:border]

In [214]:

def test_leaf(root, 
              init_node_parameters,\
              init_action_parameters,\
              node_index,\
              num_epochs,\
              max_depth, \
              train_df):


    node_parameters, action_parameters, data = \
    train_simple_optim(
        train_df,
        root,
        init_node_parameters,
        init_action_parameters,
        num_epochs=num_epochs,
        length_observation=9, 
        batch_size=128, 
        max_depth=max_depth,
        reg_coef=0.001,
        lr_np=0.01, 
        lr_ap=0.10,
        theta=0.10,
        node_index=node_index)

#       lr_np=0.001, 
#       lr_ap=0.010,

    predictions = np.array([])
    for j in range(int((len(val_df)))):
        current_index, _, _ = \
                        root.predict(batch_observation_validation[j],
                        node_parameters)
        predictions = np.append(predictions,
                               np.argmax(\
                                softmax(\
                                action_parameters[current_index])))


    _, _, _, f1 = count_metrics(predictions, batch_actions_validation)
    print('f1: %.5f' % f1)

#     f1_mean = sum(f1_vector)/len(f1_vector)
    
#     print(np.sum(node_parameters_array, axis=1))
    
#     return f1_mean, node_parameters_array[np.argmax(f1_vector)], action_parameters_array[np.argmax(f1_vector)]
    return f1, node_parameters, action_parameters, data

In [215]:
class GrowTree():
    def __init__(self,
             root,
             initial_f1,
             node_parameters,
             action_parameters,
             num_epochs,
             max_depth,
             train_df):
        
        self.current_f1 = initial_f1
        self.root = root
        self.num_epochs = num_epochs
        self.max_depth = max_depth
        self.node_parameters = node_parameters
        
        self.action_parameters = action_parameters
        self.train_df = train_df
        
        self.overall_data = pd.DataFrame({'loss':[],'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 
        
    def grow_tree(self, element, prev_element, child_type):
#         print( int(np.floor(np.log2(element.index+1))) + 2)
#         print(element.index)
#         if element.status=='optimal_leaf':
        
        if element.status=='suboptimal_leaf' and int(np.floor(np.log2(element.index+1))) + 2 < self.max_depth:
#           для начала сохраним лист на всякий случай   
            leaf_previous = element.copy()

#           посадим на место листа новый нод
            element = \
                Node(root=self.root,\
                     index=leaf_previous.index,\
                     current_f1=self.current_f1 )

#           инициализируем листья у этого нода

            element.init_node(leaf_previous.index)
            
            if child_type == 'left':
                prev_element.l_child = element
            if child_type == 'right':
                prev_element.r_child = element
                
                
            
            f1, node_parameters_best, action_parameters_best, data  = \
                test_leaf(self.root, \
                          self.node_parameters,\
                          self.action_parameters,\
                          leaf_previous.index,\
                          self.num_epochs,\
                          self.max_depth, 
                         train_df)
            
            self.overall_data = pd.concat([self.overall_data, data])

            if self.current_f1 > f1:
                
                print('save leaf with index %d , f1 = %.5f' % (leaf_previous.index, self.current_f1))
                
                leaf_previous.status = 'optimal_leaf'
                if child_type == 'left':
                    prev_element.l_child = leaf_previous
                    
                if child_type == 'right':
                    prev_element.r_child = leaf_previous
                    
            else:
                print('save split with index %d , f1 = %.5f' % (leaf_previous.index, f1))
                self.current_f1 = f1
                self.node_parameters = node_parameters_best
                self.action_parameters = action_parameters_best
            
            print('-'*50)
            process(self.root,  self.node_parameters, self.action_parameters)
                

    #         print(action_parameters[element.index])
        if child_type=='left' and prev_element.l_child.status=='node':
            self.grow_tree(element.l_child, element, 'left')
            self.grow_tree(element.r_child, element, 'right')    
                
        if child_type=='right' and  prev_element.r_child.status=='node':
            self.grow_tree(element.l_child, element, 'left')
            self.grow_tree(element.r_child, element, 'right')
            
        if child_type == None:
            self.grow_tree(element.l_child, element, 'left')
            self.grow_tree(element.r_child, element, 'right')
        
        
        
            
            
        
        
        return self.root, self.node_parameters, self.action_parameters, self.overall_data

In [161]:

# init_node_parameters = 2 * np.random.sample(\
#             ( 2**max_depth - 1,  length_observation + 1)) - 1
# init_action_parameters = 2 * np.random.sample((2**max_depth - 1, 2)) - 1

# num_epochs = 100
# max_depth = 5


# root = init_root()
# initial_f1, _, _ = test_leaf(root,init_node_parameters,init_action_parameters, 0, num_epochs, max_depth)
# # initial_f1 = 0.01
# alg = GrowTree(root, initial_f1, init_node_parameters, init_action_parameters, num_epochs=num_epochs, max_depth=max_depth)
# tree_grown, node_parameters_optimized, action_parameter_optimized = alg.grow_tree(root, None, None)


In [229]:
def fit_tree(init_node_parameters, init_action_parameters, max_depth, num_epochs, train_df):
    root = init_root()
    initial_f1, init_node_parameters, init_action_parameters, data = \
        test_leaf(root,init_node_parameters,init_action_parameters, 0, num_epochs, max_depth, train_df)
    # initial_f1 = 0.01
    alg = GrowTree(root, initial_f1, init_node_parameters, init_action_parameters, 
                   num_epochs=num_epochs, max_depth=max_depth,train_df=train_df)
    tree_grown, node_parameters_optimized, action_parameter_optimized, data = alg.grow_tree(root, None, None)
    
    return tree_grown, node_parameters_optimized, action_parameter_optimized, data 

In [85]:
def create_batch_of_trees(train_df):

    num_of_experiments = 20
    max_depth = 9
    num_epochs = 75
    
    init_node_parameters_batch = 2 * np.random.sample(\
                (num_of_experiments, 2**(max_depth-1) - 1,  length_observation + 1)) - 1
    init_action_parameters_batch = 2 * np.random.sample((num_of_experiments, 2**max_depth - 1, 2)) - 1

    history = []
    for i in range(num_of_experiments):
        tree_grown, node_parameters_optimized, action_parameter_optimized, data = \
            fit_tree(init_node_parameters_batch[i],init_action_parameters_batch[i], \
                     max_depth, num_epochs, train_df)
        history.append([tree_grown,node_parameters_optimized,action_parameter_optimized])
        
    return history

In [163]:
batch_of_trees_trained_with_oversapmling = create_batch_of_trees(train_df_oversampled)

--------------------------------------------------
Epoch:  0 	 average_loss = 0.54 	 accuracy = 0.305 	 precision = 0.26087 	 f1 = 0.40268
Epoch:  1 	 average_loss = 0.50 	 accuracy = 0.328 	 precision = 0.27434 	 f1 = 0.41892
Epoch:  2 	 average_loss = 0.53 	 accuracy = 0.352 	 precision = 0.33898 	 f1 = 0.49080
Epoch:  3 	 average_loss = 0.51 	 accuracy = 0.289 	 precision = 0.27049 	 f1 = 0.42038
Epoch:  4 	 average_loss = 0.49 	 accuracy = 0.344 	 precision = 0.29915 	 f1 = 0.45455
Epoch:  5 	 average_loss = 0.51 	 accuracy = 0.336 	 precision = 0.33043 	 f1 = 0.47205
Epoch:  6 	 average_loss = 0.49 	 accuracy = 0.352 	 precision = 0.31304 	 f1 = 0.46452
Epoch:  7 	 average_loss = 0.47 	 accuracy = 0.367 	 precision = 0.35294 	 f1 = 0.50909
Epoch:  8 	 average_loss = 0.48 	 accuracy = 0.305 	 precision = 0.27869 	 f1 = 0.43312
Epoch:  9 	 average_loss = 0.48 	 accuracy = 0.320 	 precision = 0.29661 	 f1 = 0.44586
Epoch:  10 	 average_loss = 0.47 	 accuracy = 0.336 	 precision = 0.3

Epoch:  16 	 average_loss = 0.42 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  17 	 average_loss = 0.42 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  18 	 average_loss = 0.42 	 accuracy = 0.078 	 precision = 0.07812 	 f1 = 0.14493
Epoch:  19 	 average_loss = 0.41 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  20 	 average_loss = 0.42 	 accuracy = 0.000 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_loss = 0.41 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  22 	 average_loss = 0.41 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  23 	 average_loss = 0.40 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  24 	 average_loss = 0.41 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  25 	 average_loss = 0.41 	 accuracy = 0.062 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  26 	 average_loss = 0.41 	 accuracy = 0.008 	 precision = 0.00781 	 f1 = 0.01550
Epoch:  27 	 average_

Epoch:  31 	 average_loss = 0.39 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  32 	 average_loss = 0.40 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  33 	 average_loss = 0.40 	 accuracy = 0.078 	 precision = 0.07812 	 f1 = 0.14493
Epoch:  34 	 average_loss = 0.39 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  35 	 average_loss = 0.39 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  36 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  37 	 average_loss = 0.39 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  38 	 average_loss = 0.39 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  39 	 average_loss = 0.39 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  40 	 average_loss = 0.41 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  41 	 average_loss = 0.39 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  42 	 average_

Epoch:  45 	 average_loss = 0.39 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  46 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.02381 	 f1 = 0.04651
Epoch:  47 	 average_loss = 0.37 	 accuracy = 0.062 	 precision = 0.04032 	 f1 = 0.07692
Epoch:  48 	 average_loss = 0.38 	 accuracy = 0.062 	 precision = 0.04762 	 f1 = 0.09091
Epoch:  49 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.03937 	 f1 = 0.07576
Epoch:  50 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.01587 	 f1 = 0.03125
Epoch:  51 	 average_loss = 0.37 	 accuracy = 0.031 	 precision = 0.01600 	 f1 = 0.03125
Epoch:  52 	 average_loss = 0.37 	 accuracy = 0.047 	 precision = 0.00813 	 f1 = 0.01613
Epoch:  53 	 average_loss = 0.38 	 accuracy = 0.070 	 precision = 0.06299 	 f1 = 0.11852
Epoch:  54 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  55 	 average_loss = 0.37 	 accuracy = 0.062 	 precision = 0.03226 	 f1 = 0.06250
Epoch:  56 	 average_

Epoch:  59 	 average_loss = 0.37 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  60 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  61 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  62 	 average_loss = 0.38 	 accuracy = 0.086 	 precision = 0.08594 	 f1 = 0.15827
Epoch:  63 	 average_loss = 0.37 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  64 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  65 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  66 	 average_loss = 0.37 	 accuracy = 0.000 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  67 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  68 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  69 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  70 	 average_

...

Epoch:  67 	 average_loss = 0.17 	 accuracy = 0.820 	 precision = 0.05556 	 f1 = 0.08000
Epoch:  68 	 average_loss = 0.17 	 accuracy = 0.844 	 precision = 0.05000 	 f1 = 0.09091
Epoch:  69 	 average_loss = 0.16 	 accuracy = 0.859 	 precision = 0.05882 	 f1 = 0.10000
Epoch:  70 	 average_loss = 0.15 	 accuracy = 0.844 	 precision = 0.07143 	 f1 = 0.09091
Epoch:  71 	 average_loss = 0.17 	 accuracy = 0.812 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  72 	 average_loss = 0.16 	 accuracy = 0.852 	 precision = 0.06250 	 f1 = 0.09524
Epoch:  73 	 average_loss = 0.17 	 accuracy = 0.844 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  74 	 average_loss = 0.15 	 accuracy = 0.867 	 precision = 0.09091 	 f1 = 0.10526
f1: 0.07836
save leaf with index 37 , f1 = 0.08446
--------------------------------------------------
0 node: some split
	1 node: some split
		3 node: some split
			7 optimal_leaf	not fraud

			8 node: some split
				17 optimal_leaf	fraud

				18 node: some split
					37 optimal_leaf	fra

Epoch:  0 	 average_loss = 0.16 	 accuracy = 0.852 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  1 	 average_loss = 0.16 	 accuracy = 0.883 	 precision = 0.11765 	 f1 = 0.21053
Epoch:  2 	 average_loss = 0.16 	 accuracy = 0.812 	 precision = 0.05556 	 f1 = 0.07692
Epoch:  3 	 average_loss = 0.16 	 accuracy = 0.828 	 precision = 0.05556 	 f1 = 0.08333
Epoch:  4 	 average_loss = 0.17 	 accuracy = 0.805 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  5 	 average_loss = 0.16 	 accuracy = 0.828 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  6 	 average_loss = 0.17 	 accuracy = 0.766 	 precision = 0.04348 	 f1 = 0.06250
Epoch:  7 	 average_loss = 0.16 	 accuracy = 0.812 	 precision = 0.05000 	 f1 = 0.07692
Epoch:  8 	 average_loss = 0.15 	 accuracy = 0.891 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  9 	 average_loss = 0.17 	 accuracy = 0.789 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  10 	 average_loss = 0.17 	 accuracy = 0.828 	 precision = 0.15000 	 f1 = 0.21429
Epoch:  11 	 average_loss = 0.1

Epoch:  12 	 average_loss = 0.17 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  13 	 average_loss = 0.15 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  14 	 average_loss = 0.15 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  15 	 average_loss = 0.15 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  16 	 average_loss = 0.14 	 accuracy = 0.922 	 precision = 0.20000 	 f1 = 0.16667
Epoch:  17 	 average_loss = 0.15 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  18 	 average_loss = 0.15 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  19 	 average_loss = 0.14 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  20 	 average_loss = 0.15 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_loss = 0.15 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  22 	 average_loss = 0.14 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  23 	 average_

Epoch:  23 	 average_loss = 0.14 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  24 	 average_loss = 0.15 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  25 	 average_loss = 0.15 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  26 	 average_loss = 0.14 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  27 	 average_loss = 0.16 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.16 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_loss = 0.14 	 accuracy = 0.977 	 precision = 1.00000 	 f1 = 0.57143
Epoch:  30 	 average_loss = 0.14 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  31 	 average_loss = 0.14 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  32 	 average_loss = 0.15 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  33 	 average_loss = 0.16 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  34 	 average_

Epoch:  35 	 average_loss = 0.13 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  36 	 average_loss = 0.13 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  37 	 average_loss = 0.12 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  38 	 average_loss = 0.13 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  39 	 average_loss = 0.13 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  40 	 average_loss = 0.13 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  41 	 average_loss = 0.12 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  42 	 average_loss = 0.13 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  43 	 average_loss = 0.13 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  44 	 average_loss = 0.13 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  45 	 average_loss = 0.13 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  46 	 average_

In [106]:
batch_of_trees_trained_without_oversapmling = create_batch_of_trees()

--------------------------------------------------
Epoch:  0 	 average_loss = 1.38 	 accuracy = 0.000 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  1 	 average_loss = 1.26 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  2 	 average_loss = 1.20 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  3 	 average_loss = 1.10 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  4 	 average_loss = 1.08 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  5 	 average_loss = 1.00 	 accuracy = 0.070 	 precision = 0.07031 	 f1 = 0.13139
Epoch:  6 	 average_loss = 0.99 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  7 	 average_loss = 0.95 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  8 	 average_loss = 0.95 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  9 	 average_loss = 0.91 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  10 	 average_loss = 0.91 	 accuracy = 0.016 	 precision = 0.0

Epoch:  41 	 average_loss = 0.37 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  42 	 average_loss = 0.37 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  43 	 average_loss = 0.36 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  44 	 average_loss = 0.38 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  45 	 average_loss = 0.36 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  46 	 average_loss = 0.37 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  47 	 average_loss = 0.37 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  48 	 average_loss = 0.36 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  49 	 average_loss = 0.37 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
f1: 0.07003
save split with index 1 , f1 = 0.07003
--------------------------------------------------
0 node: some split
	1 node: some split
		3 suboptimal_leaf		fraud

		4 suboptimal_leaf		fraud

	2

Epoch:  27 	 average_loss = 0.30 	 accuracy = 0.109 	 precision = 0.03419 	 f1 = 0.06557
Epoch:  28 	 average_loss = 0.31 	 accuracy = 0.086 	 precision = 0.03306 	 f1 = 0.06400
Epoch:  29 	 average_loss = 0.30 	 accuracy = 0.141 	 precision = 0.06780 	 f1 = 0.12698
Epoch:  30 	 average_loss = 0.30 	 accuracy = 0.102 	 precision = 0.02542 	 f1 = 0.04959
Epoch:  31 	 average_loss = 0.31 	 accuracy = 0.078 	 precision = 0.02479 	 f1 = 0.04839
Epoch:  32 	 average_loss = 0.30 	 accuracy = 0.102 	 precision = 0.03361 	 f1 = 0.06504
Epoch:  33 	 average_loss = 0.29 	 accuracy = 0.164 	 precision = 0.04464 	 f1 = 0.08547
Epoch:  34 	 average_loss = 0.31 	 accuracy = 0.125 	 precision = 0.05882 	 f1 = 0.11111
Epoch:  35 	 average_loss = 0.29 	 accuracy = 0.125 	 precision = 0.03448 	 f1 = 0.06667
Epoch:  36 	 average_loss = 0.30 	 accuracy = 0.109 	 precision = 0.03390 	 f1 = 0.06557
Epoch:  37 	 average_loss = 0.30 	 accuracy = 0.062 	 precision = 0.01639 	 f1 = 0.03226
Epoch:  38 	 average_

Epoch:  11 	 average_loss = 0.29 	 accuracy = 0.078 	 precision = 0.02479 	 f1 = 0.04839
Epoch:  12 	 average_loss = 0.27 	 accuracy = 0.117 	 precision = 0.01739 	 f1 = 0.03419
Epoch:  13 	 average_loss = 0.28 	 accuracy = 0.117 	 precision = 0.05042 	 f1 = 0.09600
Epoch:  14 	 average_loss = 0.28 	 accuracy = 0.148 	 precision = 0.05217 	 f1 = 0.09917
Epoch:  15 	 average_loss = 0.28 	 accuracy = 0.102 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  16 	 average_loss = 0.28 	 accuracy = 0.133 	 precision = 0.02632 	 f1 = 0.05128
Epoch:  17 	 average_loss = 0.29 	 accuracy = 0.117 	 precision = 0.03419 	 f1 = 0.06612
Epoch:  18 	 average_loss = 0.29 	 accuracy = 0.086 	 precision = 0.02500 	 f1 = 0.04878
Epoch:  19 	 average_loss = 0.28 	 accuracy = 0.125 	 precision = 0.02609 	 f1 = 0.05085
Epoch:  20 	 average_loss = 0.28 	 accuracy = 0.102 	 precision = 0.03361 	 f1 = 0.06504
Epoch:  21 	 average_loss = 0.27 	 accuracy = 0.133 	 precision = 0.00893 	 f1 = 0.01770
Epoch:  22 	 average_

Epoch:  49 	 average_loss = 0.27 	 accuracy = 0.055 	 precision = 0.03968 	 f1 = 0.07634
f1: 0.07010
save leaf with index 8 , f1 = 0.07110
--------------------------------------------------
0 node: some split
	1 node: some split
		3 node: some split
			7 node: some split
				15 optimal_leaf				fraud

				16 optimal_leaf				not fraud

			8 optimal_leaf			not fraud

		4 suboptimal_leaf		fraud

	2 suboptimal_leaf	fraud

--------------------------------------------------
Epoch:  0 	 average_loss = 0.19 	 accuracy = 0.914 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  1 	 average_loss = 0.19 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  2 	 average_loss = 0.19 	 accuracy = 0.906 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  3 	 average_loss = 0.19 	 accuracy = 0.898 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  4 	 average_loss = 0.19 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  5 	 average_loss = 0.19 	 accuracy = 0.906 	 precision = 0.10000 	 f1 = 0.14286


...

Epoch:  40 	 average_loss = 0.22 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  41 	 average_loss = 0.20 	 accuracy = 0.938 	 precision = 0.16667 	 f1 = 0.20000
Epoch:  42 	 average_loss = 0.19 	 accuracy = 0.953 	 precision = 0.33333 	 f1 = 0.25000
Epoch:  43 	 average_loss = 0.20 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  44 	 average_loss = 0.20 	 accuracy = 0.922 	 precision = 0.20000 	 f1 = 0.16667
Epoch:  45 	 average_loss = 0.20 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  46 	 average_loss = 0.21 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  47 	 average_loss = 0.19 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  48 	 average_loss = 0.19 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  49 	 average_loss = 0.21 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
f1: 0.05744
save split with index 9 , f1 = 0.05744
--------------------------------------------------
0 node: 

Epoch:  23 	 average_loss = 0.18 	 accuracy = 0.516 	 precision = 0.04839 	 f1 = 0.08824
Epoch:  24 	 average_loss = 0.18 	 accuracy = 0.531 	 precision = 0.03279 	 f1 = 0.06250
Epoch:  25 	 average_loss = 0.20 	 accuracy = 0.555 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  26 	 average_loss = 0.17 	 accuracy = 0.516 	 precision = 0.03279 	 f1 = 0.06061
Epoch:  27 	 average_loss = 0.17 	 accuracy = 0.430 	 precision = 0.01370 	 f1 = 0.02667
Epoch:  28 	 average_loss = 0.20 	 accuracy = 0.555 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_loss = 0.16 	 accuracy = 0.430 	 precision = 0.02778 	 f1 = 0.05195
Epoch:  30 	 average_loss = 0.18 	 accuracy = 0.461 	 precision = 0.01515 	 f1 = 0.02817
Epoch:  31 	 average_loss = 0.19 	 accuracy = 0.484 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  32 	 average_loss = 0.18 	 accuracy = 0.461 	 precision = 0.01562 	 f1 = 0.02817
Epoch:  33 	 average_loss = 0.18 	 accuracy = 0.477 	 precision = 0.01538 	 f1 = 0.02899
Epoch:  34 	 average_

Epoch:  5 	 average_loss = 0.19 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  6 	 average_loss = 0.18 	 accuracy = 0.938 	 precision = 0.20000 	 f1 = 0.20000
Epoch:  7 	 average_loss = 0.20 	 accuracy = 0.914 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  8 	 average_loss = 0.20 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  9 	 average_loss = 0.16 	 accuracy = 0.906 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  10 	 average_loss = 0.20 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  11 	 average_loss = 0.17 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  12 	 average_loss = 0.19 	 accuracy = 0.891 	 precision = 0.20000 	 f1 = 0.12500
Epoch:  13 	 average_loss = 0.19 	 accuracy = 0.898 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  14 	 average_loss = 0.20 	 accuracy = 0.898 	 precision = 0.11111 	 f1 = 0.13333
Epoch:  15 	 average_loss = 0.20 	 accuracy = 0.922 	 precision = 0.20000 	 f1 = 0.16667
Epoch:  16 	 average_loss 

Epoch:  42 	 average_loss = 0.20 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  43 	 average_loss = 0.19 	 accuracy = 0.914 	 precision = 0.16667 	 f1 = 0.15385
Epoch:  44 	 average_loss = 0.19 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  45 	 average_loss = 0.20 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  46 	 average_loss = 0.18 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  47 	 average_loss = 0.21 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  48 	 average_loss = 0.20 	 accuracy = 0.898 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  49 	 average_loss = 0.19 	 accuracy = 0.945 	 precision = 0.33333 	 f1 = 0.22222
f1: 0.05744
save split with index 22 , f1 = 0.05744
--------------------------------------------------
0 node: some split
	1 node: some split
		3 node: some split
			7 optimal_leaf			fraud

			8 optimal_leaf			fraud

		4 node: some split
			9 node: some split
				19 optimal_leaf				not f

Epoch:  22 	 average_loss = 0.09 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  23 	 average_loss = 0.09 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  24 	 average_loss = 0.09 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  25 	 average_loss = 0.09 	 accuracy = 0.914 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  26 	 average_loss = 0.09 	 accuracy = 0.969 	 precision = 0.50000 	 f1 = 0.33333
Epoch:  27 	 average_loss = 0.09 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.09 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_loss = 0.09 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  30 	 average_loss = 0.09 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  31 	 average_loss = 0.09 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  32 	 average_loss = 0.09 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  33 	 average_

In [178]:
def train_extra(history, train_df):

    new_history = []
    for i in range(len(history)):

        tree = history[i][0]
        
        node_parameters_optimized = history[i][1]
        action_parameter_optimized = history[i][2]

        node_parameters, action_parameters, data = \
        train_simple_optim(
            train_df,
            tree,
            node_parameters_optimized,
            action_parameter_optimized,
            num_epochs=50,
            length_observation=9, 
            batch_size=128, 
            max_depth=max_depth,
            reg_coef=0.001,
            lr_np=0.01, 
            lr_ap=0.10,
            theta=0.10,
            node_index=None)
        new_history.append((tree, node_parameters, action_parameters))
    return new_history

In [165]:
batch_observation_test = test_df.drop(['MARKED_DATE'], axis=1).values
batch_actions_test = test_df['MARKED_DATE'].values

In [166]:
def test(history):

    new_metrics = pd.DataFrame({'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 

    for i in range(len(history)):
        tree_grown = history[i][0]
        node_parameters_optimized = history[i][1]
        action_parameter_optimized = history[i][2]
        predictions = np.array([])
        for j in range(len(test_df)):
            current_index, _, _ = \
                            tree_grown.predict(batch_observation_test[j],
                            node_parameters_optimized)
            predictions = np.append(predictions,
                                   np.argmax(\
                                    softmax(\
                                    action_parameter_optimized[current_index])))

        accuracy, precision, recall, f1 = count_metrics(predictions, batch_actions_test)
        new_metrics.loc[len(new_metrics)] = [accuracy, precision,recall, f1]
    return new_metrics

In [107]:
metrics_without_os = test(batch_of_trees_trained_without_oversapmling)

In [113]:
metrics_without_os_extra_trained = train_extra(batch_of_trees_trained_without_oversapmling)

--------------------------------------------------
Epoch:  0 	 average_loss = 0.22 	 accuracy = 0.125 	 precision = 0.03478 	 f1 = 0.06667
Epoch:  1 	 average_loss = 0.23 	 accuracy = 0.086 	 precision = 0.04878 	 f1 = 0.09302
Epoch:  2 	 average_loss = 0.23 	 accuracy = 0.102 	 precision = 0.05000 	 f1 = 0.09449
Epoch:  3 	 average_loss = 0.23 	 accuracy = 0.094 	 precision = 0.02521 	 f1 = 0.04918
Epoch:  4 	 average_loss = 0.23 	 accuracy = 0.109 	 precision = 0.05042 	 f1 = 0.09524
Epoch:  5 	 average_loss = 0.22 	 accuracy = 0.188 	 precision = 0.07143 	 f1 = 0.13333
Epoch:  6 	 average_loss = 0.23 	 accuracy = 0.102 	 precision = 0.05785 	 f1 = 0.10853
Epoch:  7 	 average_loss = 0.22 	 accuracy = 0.102 	 precision = 0.02564 	 f1 = 0.04959
Epoch:  8 	 average_loss = 0.22 	 accuracy = 0.109 	 precision = 0.01724 	 f1 = 0.03390
Epoch:  9 	 average_loss = 0.23 	 accuracy = 0.078 	 precision = 0.05600 	 f1 = 0.10606
Epoch:  10 	 average_loss = 0.23 	 accuracy = 0.094 	 precision = 0.0

Epoch:  42 	 average_loss = 0.17 	 accuracy = 0.477 	 precision = 0.08333 	 f1 = 0.15190
Epoch:  43 	 average_loss = 0.17 	 accuracy = 0.438 	 precision = 0.05479 	 f1 = 0.10000
Epoch:  44 	 average_loss = 0.17 	 accuracy = 0.414 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  45 	 average_loss = 0.17 	 accuracy = 0.570 	 precision = 0.03509 	 f1 = 0.06780
Epoch:  46 	 average_loss = 0.18 	 accuracy = 0.422 	 precision = 0.02703 	 f1 = 0.05128
Epoch:  47 	 average_loss = 0.17 	 accuracy = 0.477 	 precision = 0.05714 	 f1 = 0.10667
Epoch:  48 	 average_loss = 0.17 	 accuracy = 0.461 	 precision = 0.01493 	 f1 = 0.02817
Epoch:  49 	 average_loss = 0.17 	 accuracy = 0.469 	 precision = 0.04286 	 f1 = 0.08108
--------------------------------------------------
Epoch:  0 	 average_loss = 0.13 	 accuracy = 0.883 	 precision = 0.14286 	 f1 = 0.21053
Epoch:  1 	 average_loss = 0.13 	 accuracy = 0.820 	 precision = 0.05000 	 f1 = 0.08000
Epoch:  2 	 average_loss = 0.13 	 accuracy = 0.820 	 precisio

Epoch:  34 	 average_loss = 0.02 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  35 	 average_loss = 0.02 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  36 	 average_loss = 0.02 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  37 	 average_loss = 0.02 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  38 	 average_loss = 0.02 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  39 	 average_loss = 0.02 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  40 	 average_loss = 0.02 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  41 	 average_loss = 0.02 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  42 	 average_loss = 0.02 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  43 	 average_loss = 0.02 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  44 	 average_loss = 0.02 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  45 	 average_

Epoch:  26 	 average_loss = 0.30 	 accuracy = 0.891 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  27 	 average_loss = 0.30 	 accuracy = 0.906 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.30 	 accuracy = 0.883 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_loss = 0.30 	 accuracy = 0.891 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  30 	 average_loss = 0.29 	 accuracy = 0.906 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  31 	 average_loss = 0.29 	 accuracy = 0.922 	 precision = 0.20000 	 f1 = 0.16667
Epoch:  32 	 average_loss = 0.29 	 accuracy = 0.906 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  33 	 average_loss = 0.29 	 accuracy = 0.930 	 precision = 0.12500 	 f1 = 0.18182
Epoch:  34 	 average_loss = 0.29 	 accuracy = 0.930 	 precision = 0.28571 	 f1 = 0.30769
Epoch:  35 	 average_loss = 0.29 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  36 	 average_loss = 0.30 	 accuracy = 0.922 	 precision = 0.14286 	 f1 = 0.16667
Epoch:  37 	 average_

Epoch:  18 	 average_loss = 0.39 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  19 	 average_loss = 0.39 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  20 	 average_loss = 0.39 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_loss = 0.39 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  22 	 average_loss = 0.39 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  23 	 average_loss = 0.38 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  24 	 average_loss = 0.39 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  25 	 average_loss = 0.39 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  26 	 average_loss = 0.39 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  27 	 average_loss = 0.39 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.39 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_

Epoch:  10 	 average_loss = 0.12 	 accuracy = 0.945 	 precision = 0.16667 	 f1 = 0.22222
Epoch:  11 	 average_loss = 0.14 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  12 	 average_loss = 0.12 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  13 	 average_loss = 0.13 	 accuracy = 0.891 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  14 	 average_loss = 0.12 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  15 	 average_loss = 0.13 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  16 	 average_loss = 0.14 	 accuracy = 0.898 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  17 	 average_loss = 0.13 	 accuracy = 0.930 	 precision = 0.33333 	 f1 = 0.30769
Epoch:  18 	 average_loss = 0.12 	 accuracy = 0.922 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  19 	 average_loss = 0.12 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  20 	 average_loss = 0.13 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_

In [114]:
metrics_without_os_extra_tr = test(metrics_without_os_extra_trained)

In [79]:
metrics

,accuracy,precision,recall,f1
0,0.036654,0.036617,1.000000,0.070647
1,0.036731,0.036620,1.000000,0.070652
2,0.036615,0.036615,1.000000,0.070644
3,0.648346,0.053332,0.513655,0.096631
4,0.038000,0.036666,1.000000,0.070739
5,0.037346,0.036642,1.000000,0.070694
6,0.036615,0.036615,1.000000,0.070644
7,0.871731,0.072786,0.213235,0.108527
8,0.036615,0.036615,1.000000,0.070644
9,0.043577,0.036872,1.000000,0.071122


In [83]:
new_metrics

,accuracy,precision,recall,f1
0,0.963385,0.000000,0.000000,0.000000
1,0.036731,0.036620,1.000000,0.070652
2,0.036615,0.036615,1.000000,0.070644
3,0.648346,0.053332,0.513655,0.096631
4,0.038000,0.036666,1.000000,0.070739
5,0.045923,0.036131,0.975840,0.069682
6,0.959269,0.187135,0.033613,0.056990
7,0.963385,0.000000,0.000000,0.000000
8,0.036615,0.036615,1.000000,0.070644
9,0.963269,0.000000,0.000000,0.000000


In [108]:
metrics_without_os

,accuracy,precision,recall,f1
0,0.109269,0.037682,0.950630,0.072490
1,0.481269,0.049392,0.721639,0.092457
2,0.830308,0.053894,0.219538,0.086542
3,0.963385,0.000000,0.000000,0.000000
4,0.037038,0.035630,0.970588,0.068737
5,0.912808,0.051806,0.079832,0.062836
6,0.804538,0.041925,0.198529,0.069231
7,0.040885,0.036772,1.000000,0.070936
8,0.037731,0.036621,0.998950,0.070651
9,0.938423,0.064430,0.050420,0.056570


In [110]:
num_ = 9

process(batch_of_trees_trained_without_oversapmling[num_][0],
       batch_of_trees_trained_without_oversapmling[num_][1],
       batch_of_trees_trained_without_oversapmling[num_][2])

0 node: some split
	1 node: some split
		3 node: some split
			7 optimal_leaf			fraud

			8 optimal_leaf			fraud

		4 node: some split
			9 node: some split
				19 optimal_leaf				not fraud

				20 optimal_leaf				not fraud

			10 node: some split
				21 optimal_leaf				not fraud

				22 node: some split
					45 suboptimal_leaf					not fraud

					46 suboptimal_leaf					fraud

	2 node: some split
		5 optimal_leaf		not fraud

		6 optimal_leaf		fraud



In [115]:
metrics_without_os_extra_tr

,accuracy,precision,recall,f1
0,0.963346,0.480000,0.012605,0.024565
1,0.481269,0.049392,0.721639,0.092457
2,0.963385,0.000000,0.000000,0.000000
3,0.963385,0.000000,0.000000,0.000000
4,0.963385,0.000000,0.000000,0.000000
5,0.963385,0.000000,0.000000,0.000000
6,0.804538,0.041925,0.198529,0.069231
7,0.963385,0.000000,0.000000,0.000000
8,0.963346,0.491803,0.031513,0.059230
9,0.963385,0.000000,0.000000,0.000000


In [116]:
metrics_without_os_extra_trained2 = train_extra(metrics_without_os_extra_trained)

--------------------------------------------------
Epoch:  0 	 average_loss = 0.22 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  1 	 average_loss = 0.22 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  2 	 average_loss = 0.22 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  3 	 average_loss = 0.21 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  4 	 average_loss = 0.21 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  5 	 average_loss = 0.22 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  6 	 average_loss = 0.21 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  7 	 average_loss = 0.21 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  8 	 average_loss = 0.21 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  9 	 average_loss = 0.21 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  10 	 average_loss = 0.21 	 accuracy = 0.969 	 precision = 0.0

Epoch:  42 	 average_loss = 0.16 	 accuracy = 0.445 	 precision = 0.05556 	 f1 = 0.10127
Epoch:  43 	 average_loss = 0.16 	 accuracy = 0.492 	 precision = 0.03077 	 f1 = 0.05797
Epoch:  44 	 average_loss = 0.16 	 accuracy = 0.414 	 precision = 0.05263 	 f1 = 0.09639
Epoch:  45 	 average_loss = 0.16 	 accuracy = 0.531 	 precision = 0.05000 	 f1 = 0.09091
Epoch:  46 	 average_loss = 0.17 	 accuracy = 0.469 	 precision = 0.02857 	 f1 = 0.05556
Epoch:  47 	 average_loss = 0.16 	 accuracy = 0.492 	 precision = 0.01562 	 f1 = 0.02985
Epoch:  48 	 average_loss = 0.16 	 accuracy = 0.461 	 precision = 0.04167 	 f1 = 0.08000
Epoch:  49 	 average_loss = 0.16 	 accuracy = 0.414 	 precision = 0.08642 	 f1 = 0.15730
--------------------------------------------------
Epoch:  0 	 average_loss = 0.13 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  1 	 average_loss = 0.12 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  2 	 average_loss = 0.12 	 accuracy = 0.961 	 precisio

Epoch:  34 	 average_loss = 0.02 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  35 	 average_loss = 0.02 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  36 	 average_loss = 0.02 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  37 	 average_loss = 0.02 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  38 	 average_loss = 0.02 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  39 	 average_loss = 0.02 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  40 	 average_loss = 0.02 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  41 	 average_loss = 0.02 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  42 	 average_loss = 0.02 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  43 	 average_loss = 0.02 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  44 	 average_loss = 0.02 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  45 	 average_

Epoch:  26 	 average_loss = 0.28 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  27 	 average_loss = 0.28 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.28 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_loss = 0.28 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  30 	 average_loss = 0.28 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  31 	 average_loss = 0.28 	 accuracy = 0.891 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  32 	 average_loss = 0.28 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  33 	 average_loss = 0.27 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  34 	 average_loss = 0.28 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  35 	 average_loss = 0.27 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  36 	 average_loss = 0.27 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  37 	 average_

Epoch:  18 	 average_loss = 0.38 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  19 	 average_loss = 0.38 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  20 	 average_loss = 0.38 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_loss = 0.38 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  22 	 average_loss = 0.38 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  23 	 average_loss = 0.38 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  24 	 average_loss = 0.38 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  25 	 average_loss = 0.38 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  26 	 average_loss = 0.38 	 accuracy = 0.984 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  27 	 average_loss = 0.38 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  28 	 average_loss = 0.38 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  29 	 average_

Epoch:  10 	 average_loss = 0.12 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  11 	 average_loss = 0.11 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  12 	 average_loss = 0.13 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  13 	 average_loss = 0.12 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  14 	 average_loss = 0.13 	 accuracy = 0.945 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  15 	 average_loss = 0.13 	 accuracy = 0.977 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  16 	 average_loss = 0.12 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  17 	 average_loss = 0.12 	 accuracy = 0.938 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  18 	 average_loss = 0.12 	 accuracy = 0.992 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  19 	 average_loss = 0.12 	 accuracy = 0.969 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  20 	 average_loss = 0.12 	 accuracy = 0.930 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  21 	 average_

In [119]:
metrics_without_os_extra_tr2 = test(metrics_without_os_extra_trained2)

In [120]:
metrics_without_os_extra_tr2

,accuracy,precision,recall,f1
0,0.963385,0.000000,0.000000,0.000000
1,0.481269,0.049392,0.721639,0.092457
2,0.963385,0.000000,0.000000,0.000000
3,0.963385,0.000000,0.000000,0.000000
4,0.963385,0.000000,0.000000,0.000000
5,0.963385,0.000000,0.000000,0.000000
6,0.804538,0.041925,0.198529,0.069231
7,0.963385,0.000000,0.000000,0.000000
8,0.963423,0.508475,0.031513,0.059347
9,0.963385,0.000000,0.000000,0.000000


In [124]:
num_ = 8

process(metrics_without_os_extra_trained2[num_][0],
       metrics_without_os_extra_trained2[num_][1],
       metrics_without_os_extra_trained2[num_][2])

0 node: some split
	1 node: some split
		3 node: some split
			7 optimal_leaf			not fraud

			8 node: some split
				17 node: some split
					35 suboptimal_leaf					not fraud

					36 suboptimal_leaf					not fraud

				18 node: some split
					37 suboptimal_leaf					fraud

					38 suboptimal_leaf					not fraud

		4 node: some split
			9 optimal_leaf			fraud

			10 optimal_leaf			fraud

	2 optimal_leaf	fraud



In [125]:
num_ = 8

best_tree_optimized = metrics_without_os_extra_trained2[num_][0]
best_node_parameters_optimized = metrics_without_os_extra_trained2[num_][1]
best_action_parameters_optimized = metrics_without_os_extra_trained2[num_][2]

In [169]:
class InterpretableNode:
    """
    Class for Node with split
    """
    def __init__(
        self,
        status=None,
        root=None,
        index=None,
        split_var_ind=None,
        split_value=None,
        b_value=None
        
    ):
        
        # Setting index of elemnet
        self.index = index if index else 0
        self.depth = int(np.floor(np.log2(self.index + 2)))
        self.status = status if status else "node"
        self.split_var_ind = split_var_ind if split_var_ind else None
        self.split_value = split_value if split_value else None
        self.b_value = b_value if b_value else None
        # Init root with None
        self.root = root
        
    def init_node(self, index):
        # Init childs with Leafs
        self.l_child = \
            InterpretableNode(
                root=self.root,\
                index=2*index + 1)
        self.r_child = \
            InterpretableNode(
                root=self.root,\
                index=2*index + 2)
        
        
    
    def predict(self, 
                observations: list):

        current = self
        index = 0

        # проходим по дереву пока не упираемся в leaf
        while current.status == 'node':

            if self.b_value + observations[self.split_var_ind]*self.split_value > 0:
#                 print('RIGHT')
                current = current.l_child
                index = 2*index + 1
            else:
#                 print('LEFT')
                current = current.r_child
                index = 2*index + 2
                
        
        return index               
    


In [190]:
translated_root = InterpretableNode()
translated_root.root = translated_root
# translated_root.

In [233]:
best_node_parameters_optimized.shape

(63, 10)

In [243]:
best_node_parameters_optimized[-1]

array([ 0.11692329, -0.09331371, -0.01503134, -0.13480852,  0.16046669,
       -0.15198329, -0.01179237, -0.07940498,  0.07032018,  0.17067319])

In [242]:
np.min(best_node_parameters_optimized[:-1], axis=1)

array([-0.16431786, -0.1750589 , -0.18359974, -0.17152749, -0.18868077,
       -0.07260428, -0.17541705, -0.18965226, -0.14733006, -0.09344811,
       -0.12233599, -0.1272497 , -0.173536  , -0.17998783, -0.18055012,
       -0.12520724, -0.14009266, -0.17436656, -0.18719701, -0.15233557,
       -0.16853941, -0.16815322, -0.10694014, -0.12832729, -0.16335815,
       -0.17332603, -0.17296871, -0.17907197, -0.09846275, -0.15403245,
       -0.02203249, -0.16892023, -0.07853391, -0.15139947, -0.17749762,
       -0.10775327, -0.18670107, -0.12812117, -0.18129324, -0.07543405,
       -0.18983097, -0.17457748, -0.07480435, -0.15604244, -0.12102722,
       -0.12124135, -0.18336268, -0.13810282, -0.17365148, -0.10822542,
       -0.17187101, -0.19022867, -0.19021916, -0.16359577, -0.1423932 ,
       -0.15237887, -0.15158122, -0.14452021, -0.14901052, -0.1469546 ,
       -0.09618905, -0.18998561])

In [241]:
np.max(best_node_parameters_optimized[:-1], axis=1)

array([0.17941514, 0.11439093, 0.08994108, 0.18075347, 0.16313251,
       0.16333578, 0.10214071, 0.18395217, 0.05247288, 0.18221632,
       0.16757958, 0.1452063 , 0.12832028, 0.16556126, 0.19001242,
       0.17544075, 0.14176902, 0.1183911 , 0.16304783, 0.11840048,
       0.15375383, 0.14546474, 0.12665728, 0.13957955, 0.13876705,
       0.16770327, 0.16593238, 0.06315864, 0.15880065, 0.17285303,
       0.18315312, 0.13140248, 0.16155528, 0.17515049, 0.13230481,
       0.0985622 , 0.17629079, 0.18320898, 0.09258907, 0.18334175,
       0.18840573, 0.14545428, 0.17392325, 0.07760589, 0.15837721,
       0.17128921, 0.17184402, 0.16967666, 0.16900181, 0.16042867,
       0.17649213, 0.13020972, 0.19058962, 0.17648417, 0.17446701,
       0.0972492 , 0.13173711, 0.17855863, 0.16481825, 0.12762863,
       0.16684827, 0.17288435])

In [170]:
def translate(element, translated_element, node_parameters, action_parameters):
    translated_element.index = element.index
    
    if element.status!='node':
#         print('there is a leaf')
        translated_element.status = 'leaf'
        
        
    if element.status=='node':
#         print('there is a node')
        translated_element.split_var_ind = np.argmax(node_parameters[element.index][:-1])
        translated_element.split_value = np.max(node_parameters[element.index][:-1])
        translated_element.b_value = node_parameters[element.index][-1]
        
        translated_element.init_node(translated_element.index)
        
        translate(element.l_child, translated_element.l_child, node_parameters, action_parameters)
        translate(element.r_child, translated_element.r_child, node_parameters, action_parameters)

In [193]:
translate(best_tree_optimized,translated_root, best_node_parameters_optimized, best_action_parameters_optimized)

In [194]:
process(translated_root, best_node_parameters_optimized, best_action_parameters_optimized)

0 node: some split
	1 node: some split
		3 node: some split
			7 leaf			not fraud

			8 node: some split
				17 node: some split
					35 leaf					not fraud

					36 leaf					not fraud

				18 node: some split
					37 leaf					fraud

					38 leaf					not fraud

		4 node: some split
			9 leaf			fraud

			10 leaf			fraud

	2 leaf	fraud



In [179]:
def process_translated_tree(element, action_parameters):
#     print(np.log2(element.index + 1))
    print('\t'*int(np.floor(np.log2(element.index+1))) + str(element.index) + ' ' + element.status, end='')
    if element.status!='node':
#         print('\t'*int(np.floor(np.log2(element.index+1))) , end='')
        print('\t', end='')
        to_actions(action_parameters[element.index])
        print(to_actions(action_parameters[element.index]))
        print()
#         print(action_parameters[element.index])
    if element.status=='node':
        print(' :' + columns_names[element.split_var_ind],end='')
        if element.split_value > 0:
            print(': > %.2f' % ( - element.b_value/ element.split_value))
        else:
            print(': < %.2f' % (- element.b_value/ element.split_value))
        
        process_translated_tree(element.l_child, action_parameters)
        process_translated_tree(element.r_child, action_parameters)

In [245]:
process_translated_tree(translated_root, best_action_parameters_optimized)

0 node :PDC_CARD_PRES: > -0.66
	1 node :AMOUNT_INST: > 1.52
		3 node :PDC_CARD_PRES: > 0.31
			7 leaf	not fraud

			8 node :CVV2_PRES: > 0.16
				17 node :AMOUNT: > 0.96
					35 leaf	not fraud

					36 leaf	not fraud

				18 node :DATE_TIME: > 0.46
					37 leaf	fraud

					38 leaf	not fraud

		4 node :PIN_PRES: > -0.58
			9 leaf	fraud

			10 leaf	fraud

	2 leaf	fraud



In [171]:
def translate_history(history):
    new_history = []
    
    for i in range(len(history)):
        translated_root = InterpretableNode()
        translated_root.root = translated_root

        translate(history[i][0], translated_root,\
                  history[i][1], history[i][2])
        
        new_history.append((translated_root, history[i][2]))
        
    return new_history

In [204]:
batch_with_translated_trees = translate_history(metrics_without_os_extra_trained2)

In [175]:
def test_translated_trees(history):

    new_metrics = pd.DataFrame({'accuracy':[], 'precision':[], 'recall':[], 'f1':[]}) 

    for i in range(len(history)):
        tree_grown = history[i][0]
        action_parameter_optimized = history[i][1]
        predictions = np.array([])
        for j in range(len(test_df)):
            current_index = \
                            tree_grown.predict(batch_observation_test[j])
            predictions = np.append(predictions,
                                   np.argmax(\
                                    softmax(\
                                    action_parameter_optimized[current_index])))

        accuracy, precision, recall, f1 = count_metrics(predictions, batch_actions_test)
        new_metrics.loc[len(new_metrics)] = [accuracy, precision,recall, f1]
    return new_metrics

In [208]:
metrics_translated_tree = test_translated_trees(batch_with_translated_trees)

In [209]:
metrics_translated_tree

,accuracy,precision,recall,f1
0,0.963385,0.000000,0.0,0.000000
1,0.036615,0.036615,1.0,0.070644
2,0.963385,0.000000,0.0,0.000000
3,0.963385,0.000000,0.0,0.000000
4,0.963385,0.000000,0.0,0.000000
5,0.963385,0.000000,0.0,0.000000
6,0.036615,0.036615,1.0,0.070644
7,0.963385,0.000000,0.0,0.000000
8,0.963385,0.000000,0.0,0.000000
9,0.963385,0.000000,0.0,0.000000


In [246]:
for i in range(len(batch_with_translated_trees)):
    print(i)
    process_translated_tree(batch_with_translated_trees[i][0], batch_with_translated_trees[i][1])
    print('-'*50)
    print()

0
0 node :DATE_TIME: > -0.67
	1 node :AMOUNT: > 1.00
		3 node :PIN_PRES: > -0.54
			7 node :CVV2_PRES: > -0.50
				15 leaf	fraud

				16 leaf	not fraud

			8 leaf	not fraud

		4 leaf	fraud

	2 node :AUTH_RESP_CODE_FM: > 1.03
		5 node :AMOUNT: > 0.04
			11 node :CVV2_PRES: > -0.42
				23 node :PDC_CARD_CAPTURE_CAP: > 0.43
					47 leaf	fraud

					48 leaf	fraud

				24 leaf	fraud

			12 leaf	fraud

		6 leaf	fraud

--------------------------------------------------

1
0 node :DATE_TIME: > -1.19
	1 leaf	fraud

	2 node :PDC_CARD_PRES: > -0.63
		5 leaf	fraud

		6 node :AMOUNT_INST: > -0.98
			13 leaf	fraud

			14 leaf	not fraud

--------------------------------------------------

2
0 node :AMOUNT_INST: > 0.04
	1 node :PIN_PRES: > -0.07
		3 node :CARD_EXPR_DATE: > 0.29
			7 leaf	not fraud

			8 node :AMOUNT: > 0.70
				17 node :PIN_PRES: > -1.35
					35 leaf	fraud

					36 leaf	not fraud

				18 node :PIN_PRES: > 0.05
					37 leaf	fraud

					38 leaf	fraud

		4 leaf	fraud

	2 node :PIN_PRES: 

In [167]:
metrics_with_os = test(batch_of_trees_trained_with_oversapmling)

In [168]:
metrics_with_os

,accuracy,precision,recall,f1
0,0.035231,0.035231,1.000000,0.068064
1,0.035231,0.035231,1.000000,0.068064
2,0.858962,0.032937,0.105895,0.050246
3,0.035308,0.035233,1.000000,0.068069
4,0.112654,0.034657,0.900655,0.066745
5,0.035308,0.035233,1.000000,0.068069
6,0.604192,0.052249,0.597162,0.096091
7,0.035385,0.035236,1.000000,0.068074
8,0.333538,0.025829,0.487991,0.049062
9,0.035269,0.035232,1.000000,0.068066


In [173]:
translated_history_with_oversampling = translate_history(batch_of_trees_trained_with_oversapmling)

In [176]:
metrics_translated_tree_with_oversampling = test_translated_trees(translated_history_with_oversampling)

In [177]:
metrics_translated_tree_with_oversampling

,accuracy,precision,recall,f1
0,0.035231,0.035231,1.000000,0.068064
1,0.035231,0.035231,1.000000,0.068064
2,0.035231,0.035231,1.000000,0.068064
3,0.035231,0.035231,1.000000,0.068064
4,0.964769,0.000000,0.000000,0.000000
5,0.065231,0.034956,0.959607,0.067455
6,0.035231,0.035231,1.000000,0.068064
7,0.035231,0.035231,1.000000,0.068064
8,0.964154,0.416667,0.043668,0.079051
9,0.866192,0.032129,0.096070,0.048153


In [206]:
metrics_translated_tree_with_oversampling[metrics_translated_tree_with_oversampling.accuracy > 0.95]

,accuracy,precision,recall,f1
4,0.964769,0.000000,0.000000,0.000000
8,0.964154,0.416667,0.043668,0.079051
14,0.964769,0.000000,0.000000,0.000000
19,0.964769,0.000000,0.000000,0.000000


In [180]:
process_translated_tree(translated_history_with_oversampling[8][0], translated_history_with_oversampling[8][1])

0 node :DATE_TIME: > 1.05
	1 leaf	fraud

	2 node :PDC_CARD_CAPTURE_CAP: > -0.29
		5 node :AMOUNT_INST: > -0.16
			11 node :AUTH_RESP_CODE_FM: > -0.71
				23 node :DATE_TIME: > -0.34
					47 node :CVV2_PRES: > 1.29
						95 node :CARD_EXPR_DATE: > 1.42
							191 leaf	not fraud

							192 leaf	fraud

						96 leaf	fraud

					48 leaf	fraud

				24 leaf	fraud

			12 leaf	fraud

		6 leaf	not fraud



In [194]:

def count_complexity_matrix(tree_traslated, action_parameters, batch_observation, true_values):
    
    predictions = np.array([])
    for j in range(int(len(batch_observation))):
        current_index = \
                        tree_traslated.predict(batch_observation[j])
        predictions = np.append(predictions,
                               np.argmax(\
                                softmax(\
                                action_parameters[current_index])))


        
    TP = ((predictions == 1) & (true_values == 1)).sum()
    TN = ((predictions == 0) & (true_values == 0)).sum()
    FP = ((predictions == 1) & (true_values == 0)).sum()
    FN = ((predictions == 0 ) & (true_values == 1 )).sum()        
        
    return TP, TN, FP, FN

In [195]:
TP, TN, FP, FN = count_complexity_matrix(translated_history_with_oversampling[8][0],\
                                         translated_history_with_oversampling[8][1],\
                                         batch_observation_test,\
                                         batch_actions_test) 

In [200]:
complexity_matrix = pd.DataFrame({'Predicted':[],'True Not fraud':[], 'True fraud':[]})

In [204]:
complexity_matrix.loc[0] = ['Pred Not Fraud', TN, FN]
complexity_matrix.loc[1] = ['Pred Fraud', FP, TP]


In [205]:
complexity_matrix

,Predicted,True Not fraud,True fraud
0,Pred Not Fraud,25028,876
1,Pred Fraud,56,40


In [210]:
Specificity = TN / (TN + FP)
Specificity

0.9977675011959815

In [209]:
Sensetivity = TP / (TP + FN)
Sensetivity

0.043668122270742356

In [232]:

max_depth = 7
num_epochs = 75
num_ = 10
init_node_parameters = 2 * np.random.sample(\
            ( num_,2**(max_depth-1) - 1,  length_observation + 1)) - 1
init_action_parameters = 2 * np.random.sample((num_, 2**max_depth - 1, 2)) - 1

for i in range(num_):

    tree_grown, node_parameters_optimized, action_parameter_optimized, data = \
        fit_tree(init_node_parameters[i],init_action_parameters[i], \
                 max_depth, num_epochs, train_df)
    name = str(i) + '.csv'

    data.to_csv('log_4/' + name, sep='\t')




--------------------------------------------------
Epoch:  0 	 average_loss = 0.67 	 accuracy = 0.070 	 precision = 0.07031 	 f1 = 0.13139
Epoch:  1 	 average_loss = 0.66 	 accuracy = 0.062 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  2 	 average_loss = 0.66 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  3 	 average_loss = 0.62 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  4 	 average_loss = 0.61 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  5 	 average_loss = 0.61 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  6 	 average_loss = 0.60 	 accuracy = 0.070 	 precision = 0.07031 	 f1 = 0.13139
Epoch:  7 	 average_loss = 0.60 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  8 	 average_loss = 0.60 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  9 	 average_loss = 0.58 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  10 	 average_loss = 0.58 	 accuracy = 0.062 	 precision = 0.0

Epoch:  16 	 average_loss = 0.46 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  17 	 average_loss = 0.46 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  18 	 average_loss = 0.46 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  19 	 average_loss = 0.46 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  20 	 average_loss = 0.47 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  21 	 average_loss = 0.45 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  22 	 average_loss = 0.46 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  23 	 average_loss = 0.45 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  24 	 average_loss = 0.47 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  25 	 average_loss = 0.47 	 accuracy = 0.008 	 precision = 0.00781 	 f1 = 0.01550
Epoch:  26 	 average_loss = 0.46 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  27 	 average_

Epoch:  31 	 average_loss = 0.42 	 accuracy = 0.164 	 precision = 0.05357 	 f1 = 0.10084
Epoch:  32 	 average_loss = 0.41 	 accuracy = 0.180 	 precision = 0.01869 	 f1 = 0.03670
Epoch:  33 	 average_loss = 0.41 	 accuracy = 0.180 	 precision = 0.02830 	 f1 = 0.05405
Epoch:  34 	 average_loss = 0.41 	 accuracy = 0.195 	 precision = 0.03738 	 f1 = 0.07207
Epoch:  35 	 average_loss = 0.42 	 accuracy = 0.172 	 precision = 0.07018 	 f1 = 0.13115
Epoch:  36 	 average_loss = 0.42 	 accuracy = 0.164 	 precision = 0.03636 	 f1 = 0.06957
Epoch:  37 	 average_loss = 0.41 	 accuracy = 0.180 	 precision = 0.01869 	 f1 = 0.03670
Epoch:  38 	 average_loss = 0.41 	 accuracy = 0.203 	 precision = 0.06422 	 f1 = 0.12069
Epoch:  39 	 average_loss = 0.42 	 accuracy = 0.148 	 precision = 0.01818 	 f1 = 0.03540
Epoch:  40 	 average_loss = 0.41 	 accuracy = 0.172 	 precision = 0.02752 	 f1 = 0.05357
Epoch:  41 	 average_loss = 0.41 	 accuracy = 0.156 	 precision = 0.02703 	 f1 = 0.05263
Epoch:  42 	 average_

Epoch:  46 	 average_loss = 0.43 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  47 	 average_loss = 0.43 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  48 	 average_loss = 0.42 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  49 	 average_loss = 0.42 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  50 	 average_loss = 0.43 	 accuracy = 0.070 	 precision = 0.07031 	 f1 = 0.13139
Epoch:  51 	 average_loss = 0.42 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  52 	 average_loss = 0.43 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  53 	 average_loss = 0.43 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  54 	 average_loss = 0.43 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  55 	 average_loss = 0.43 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  56 	 average_loss = 0.44 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  57 	 average_

Epoch:  60 	 average_loss = 0.41 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  61 	 average_loss = 0.41 	 accuracy = 0.070 	 precision = 0.07031 	 f1 = 0.13139
Epoch:  62 	 average_loss = 0.40 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  63 	 average_loss = 0.41 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  64 	 average_loss = 0.40 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  65 	 average_loss = 0.41 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  66 	 average_loss = 0.41 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  67 	 average_loss = 0.40 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  68 	 average_loss = 0.41 	 accuracy = 0.062 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  69 	 average_loss = 0.40 	 accuracy = 0.062 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  70 	 average_loss = 0.40 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  71 	 average_

...

Epoch:  59 	 average_loss = 0.39 	 accuracy = 0.008 	 precision = 0.00781 	 f1 = 0.01550
Epoch:  60 	 average_loss = 0.39 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  61 	 average_loss = 0.39 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.06061
Epoch:  62 	 average_loss = 0.39 	 accuracy = 0.062 	 precision = 0.06250 	 f1 = 0.11765
Epoch:  63 	 average_loss = 0.39 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  64 	 average_loss = 0.39 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  65 	 average_loss = 0.39 	 accuracy = 0.055 	 precision = 0.05469 	 f1 = 0.10370
Epoch:  66 	 average_loss = 0.39 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  67 	 average_loss = 0.39 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  68 	 average_loss = 0.39 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  69 	 average_loss = 0.39 	 accuracy = 0.008 	 precision = 0.00781 	 f1 = 0.01550
Epoch:  70 	 average_

Epoch:  71 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  72 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  73 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  74 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
f1: 0.07177
save split with index 8 , f1 = 0.07177
--------------------------------------------------
0 node: some split
	1 node: some split
		3 node: some split
			7 node: some split
				15 node: some split
					31 suboptimal_leaf	fraud

					32 suboptimal_leaf	fraud

				16 node: some split
					33 suboptimal_leaf	not fraud

					34 suboptimal_leaf	not fraud

			8 node: some split
				17 suboptimal_leaf	not fraud

				18 suboptimal_leaf	not fraud

		4 suboptimal_leaf	fraud

	2 suboptimal_leaf	fraud

--------------------------------------------------
Epoch:  0 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.03125 	 f1 = 0.0606

Epoch:  1 	 average_loss = 0.38 	 accuracy = 0.070 	 precision = 0.06299 	 f1 = 0.11852
Epoch:  2 	 average_loss = 0.38 	 accuracy = 0.078 	 precision = 0.07812 	 f1 = 0.14493
Epoch:  3 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03150 	 f1 = 0.06107
Epoch:  4 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.02362 	 f1 = 0.04615
Epoch:  5 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  6 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  7 	 average_loss = 0.38 	 accuracy = 0.031 	 precision = 0.02362 	 f1 = 0.04615
Epoch:  8 	 average_loss = 0.38 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  9 	 average_loss = 0.38 	 accuracy = 0.062 	 precision = 0.05512 	 f1 = 0.10448
Epoch:  10 	 average_loss = 0.38 	 accuracy = 0.023 	 precision = 0.01575 	 f1 = 0.03101
Epoch:  11 	 average_loss = 0.38 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  12 	 average_loss = 0.

Epoch:  13 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  14 	 average_loss = 0.38 	 accuracy = 0.039 	 precision = 0.03906 	 f1 = 0.07519
Epoch:  15 	 average_loss = 0.38 	 accuracy = 0.016 	 precision = 0.01562 	 f1 = 0.03077
Epoch:  16 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.04688 	 f1 = 0.08955
Epoch:  17 	 average_loss = 0.38 	 accuracy = 0.047 	 precision = 0.03937 	 f1 = 0.07576
Epoch:  18 	 average_loss = 0.38 	 accuracy = 0.023 	 precision = 0.02344 	 f1 = 0.04580
Epoch:  19 	 average_loss = 0.37 	 accuracy = 0.047 	 precision = 0.03175 	 f1 = 0.06154
Epoch:  20 	 average_loss = 0.38 	 accuracy = 0.008 	 precision = 0.00781 	 f1 = 0.01550
Epoch:  21 	 average_loss = 0.38 	 accuracy = 0.961 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  22 	 average_loss = 0.38 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  23 	 average_loss = 0.38 	 accuracy = 0.953 	 precision = 0.00000 	 f1 = 0.00000
Epoch:  24 	 average_

KeyboardInterrupt: 

In [ ]:
# print('accuracy : %.5f, precision: %.5f, recall: %.5f, f1: %.5f' % (f1,precision,recall ,f1))